In [15]:
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen
import json
import time
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotVisibleException
import pandas as pd
import numpy as np

In [2]:
store_number_lst = ['21726749', '34582591','1670762252']
see_more_xpath_lst = ['//*[@id="app-root"]/div/div/div[2]/div[5]/div/div[4]/div[2]/a','//*[@id="app-root"]/div/div/div[2]/div[5]/div/div[8]/div[2]/a','//*[@id="app-root"]/div/div/div[2]/div[5]/div/div[4]/div[2]/a']

In [3]:
store_number = store_number_lst[1] 
options = webdriver.ChromeOptions()
# options.add_argument('headless') #창 안 뜨게 설정
review_driver = webdriver.Chrome('C:/Users/daye/practice/chromedriver.exe',options=options)# 크롬 경로
review_driver.implicitly_wait(3)

#url 접속
review_url = 'https://pcmap.place.naver.com/restaurant/'+store_number+'/home?from=map&amp;fromPanelNum=2&amp'
review_driver.get(review_url)
time.sleep(5)

In [4]:
review_driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

In [5]:
#카페명 찾기

html = review_driver.page_source
cafe = BeautifulSoup(html, 'lxml')
cafe = cafe.find('span', class_='_3XamX').text

In [10]:
category_xpath_list = ['//*[@id="app-root"]/div/div/div[2]/div[5]/div[4]/div[5]/div[1]/div[1]/div/a[2]',
                       '//*[@id="app-root"]/div/div/div[2]/div[5]/div[4]/div[5]/div[1]/div[1]/div/a[3]',
                       '//*[@id="app-root"]/div/div/div[2]/div[5]/div[4]/div[5]/div[1]/div[1]/div/a[4]']
#맛, 서비스,분위기 순 클릭

def get_category_review(x):
    
        button = review_driver.find_element_by_xpath(category_xpath_list[x])
        button.click()
    
        html = review_driver.page_source
        soup = BeautifulSoup(html, 'lxml')
    
        results = review_driver.find_elements_by_class_name('highlight')
        results_list = [result.text for result in results]
        
        return results_list

In [16]:
cafe_col = np.repeat(cafe, len(get_category_review(0) + get_category_review(1) + get_category_review(2)))

array(['투썸플레이스 종각역점', '투썸플레이스 종각역점', '투썸플레이스 종각역점', '투썸플레이스 종각역점',
       '투썸플레이스 종각역점', '투썸플레이스 종각역점', '투썸플레이스 종각역점', '투썸플레이스 종각역점',
       '투썸플레이스 종각역점', '투썸플레이스 종각역점', '투썸플레이스 종각역점', '투썸플레이스 종각역점',
       '투썸플레이스 종각역점', '투썸플레이스 종각역점', '투썸플레이스 종각역점', '투썸플레이스 종각역점',
       '투썸플레이스 종각역점', '투썸플레이스 종각역점', '투썸플레이스 종각역점', '투썸플레이스 종각역점',
       '투썸플레이스 종각역점', '투썸플레이스 종각역점', '투썸플레이스 종각역점'], dtype='<U11')

In [19]:
df_taste = pd.DataFrame({"항목": '맛', "내용": get_category_review(0)})
df_service = pd.DataFrame({"항목": '서비스', "내용": get_category_review(1)})
df_atmosphere = pd.DataFrame({"항목": '만족도', "내용": get_category_review(2)})
df = pd.concat([df_taste,df_service,df_atmosphere], ignore_index=True)
df.insert(1, '지점', cafe_col)

In [20]:
df

,항목,지점,내용
0,맛,투썸플레이스 종각역점,직원들 짱친절하고
1,맛,투썸플레이스 종각역점,직원분도 짱 친절하셨어요!
2,맛,투썸플레이스 종각역점,마끼아또가 라떼에 카라멜드리즐한것같이 맛없어서 왠만하면 참으려다 너무 맛없어서
3,맛,투썸플레이스 종각역점,먼저 직원분들 친절하시고
4,맛,투썸플레이스 종각역점,초콜렛이 진하면서 맛있었습니다
5,맛,투썸플레이스 종각역점,음료 맛도 굿굿
6,맛,투썸플레이스 종각역점,맛있어요.
7,맛,투썸플레이스 종각역점,맛있어요
8,맛,투썸플레이스 종각역점,맛있어요
9,서비스,투썸플레이스 종각역점,직원들은 친절한데
